# Name: Margaret Nguyen

# Data Aggregation: Massachusetts Person Level Crash Data and ACS Data

**Assignment: Retrieve person level crash data from 2017 to 2021 for Massachusetts. Try to find a way to organize the data by municipality so that we can later merge it with county subdivision data from the ACS 5-year estimates. You can access the Massachusetts crash data [here](https://apps.impact.dot.state.ma.us/cdp/home). Click on "Data Extraction" and search for crashes; they should be available by year.**

**Here is the link to the Massachusetts Law Enforcement Crash Report Data Dictionary: [link](https://www.umasstransportationcenter.org/images/umtc/UMassSafe/Massachusetts%20Crash%20Report%20Data%20Dictionary.pdf).**

## Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Carlisle-Local-Crash-Analysis/blob/main/1.%20Municupal%20Crash%20Data%20Aggregation%20.ipynb).

In [6]:
# Import necessary libraries 
import requests
import json, csv
import timeit
from pathlib import Path  

# Import data handling libraries 
import numpy as np
import pandas as pd
import requests
import json

# Import graphing libraries 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from pathlib import Path  

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.6f}'.format

## Set Global Variables

In [7]:
# Set Global Variables

# Set the time period in which to look at 
START_YEAR = 2017
END_YEAR = 2021

## Query ACS Data for Municipalities 

In [8]:
# Query ACS data for Municipalities 

# Census API
HOST = 'https://api.census.gov/data'

# Year to get data for 
YEAR = '{}'.format(END_YEAR)# 2005,2010,2015,2020  

# Survey to Access data from (ACS 5 year estimates)
DATA_SET = 'acs/acs5'
BASE_URL = '/'.join([HOST, YEAR, DATA_SET])

# Create an empty dictionary for predicates 
predicates = {}

# VARIABLES
# Population Estimate:  B01003_001E
# Population Margin of error: B01003_001M
# Bike to work Estimate: B08006_014E
# Bike to work Margin of error: B08006_014M
# Walk to work Estimate:B08006_015E
# Walk to work Margin of error:B08006_015M
# Drive to work alone Estimate: B08006_003E
# Drive to work alone Margin of error: B08006_003M
# Carpool to work Estimate:B08006_004E
# Carpool to work Margin: 
# Public transit to work Estimate: B08006_008E
# Public transit to work Margine of Error: B08006_008E
# Other to Work 
# Poverty Estimate:
# Poverty Margin of error:

get_vars = ['NAME',
            'B01001_001E',
            'B08006_014E',
            'B08006_014M',
            'B08006_015E',
            'B08006_015M',
            'B08006_003E',
            'B08006_003M',
            'B08006_004E',
            'B08006_004M',
            'B08006_008E',
            'B08006_008M']

predicates['get']=','.join(get_vars)

# Set sub geographies to get data for ('county','Place','county subdivision') * means get all 
predicates['for']='county subdivision:*'

# Set geography that contains sub geographies  (25 = Massachusetts)
predicates['in']='state:25'

# Assemble the API query 
r = requests.get(BASE_URL, params = predicates)

# Print the query URL 
print(BASE_URL,predicates)

https://api.census.gov/data/2021/acs/acs5 {'get': 'NAME,B01001_001E,B08006_014E,B08006_014M,B08006_015E,B08006_015M,B08006_003E,B08006_003M,B08006_004E,B08006_004M,B08006_008E,B08006_008M', 'for': 'county subdivision:*', 'in': 'state:25'}


In [9]:
# Print the length and first thousand characters to see what you got 
print(len(r.text))
print(r.text[0:1000])

47736
[["NAME","B01001_001E","B08006_014E","B08006_014M","B08006_015E","B08006_015M","B08006_003E","B08006_003M","B08006_004E","B08006_004M","B08006_008E","B08006_008M","state","county","county subdivision"],
["County subdivisions not defined, Barnstable County, Massachusetts","0","0","13","0","13","0","13","0","13","0","13","25","001","00000"],
["Barnstable Town city, Barnstable County, Massachusetts","48556","35","52","843","321","18901","926","2649","482","303","136","25","001","03690"],
["Bourne town, Barnstable County, Massachusetts","20364","0","25","182","64","8471","673","722","288","84","55","25","001","07175"],
["Brewster town, Barnstable County, Massachusetts","10282","66","70","1","3","3733","445","153","88","24","29","25","001","07980"],
["Chatham town, Barnstable County, Massachusetts","6554","0","19","124","74","1666","252","196","96","93","102","25","001","12995"],
["Dennis town, Barnstable County, Massachusetts","14664","8","16","147","108","4930","515","658","173","37

In [10]:
# Place the Queried ACS data into a data frame 

# Set the column names to the first row of data from the query 
column_names=r.json()[0:1][0]

# Set the data to everything after the first row and convert to an array to flatten it
ACS_DATA= r.json()[1:]
ACS_data = np.array(ACS_DATA)

# Create the pandas data frame 
ACS_MUNI_DF = pd.DataFrame(columns=column_names , data = ACS_data)

# Reset the index of the data frame
ACS_MUNI_DF.reset_index()

# Rename the columns 
ACS_MUNI_DF.rename(columns ={"B01001_001E":"POPULATION",
                    "B08006_014E":'BIKE_TO_WORK_EST',
                    "B08006_014M":"BIKE_TO_WORK_MARG",
                    "B08006_015E":"WALK_TO_WORK_EST",
                    "B08006_015M":"WALK_TO_WORK_MARG",
                    'B08006_003E':"DRIVE_SOLO_TO_WORK_EST",
                    'B08006_003M':"DRIVE_SOLO_TO_WORK_MARG",
                    'B08006_004E':"CARPOOL_TO_WORK_EST",
                    'B08006_004M':"CARPOOL_TO_WORK_MARG",
                    'B08006_008E':"PUBTRANS_TO_WORK_EST",
                    'B08006_008M':"PUBTRANS_TO_WORK_MARG",
                    "county subdivision":"county_subdivision"}, inplace=True)

# Convert the NAME column to strings 
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF["NAME"].astype(str)

# Remove Massachusetts from NAME This is over redundant since all data will be from Massachusetts
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF.NAME.replace({', Massachusetts':''}, regex=True)

# Create separate Name variables for county and municipality 
ACS_MUNI_DF[['MUNI_NAME','COUNTY_NAME']]=ACS_MUNI_DF['NAME'].str.split(',',expand=True)

# Convert the variables to integers
ACS_MUNI_DF[["POPULATION",
                "BIKE_TO_WORK_EST",
                "BIKE_TO_WORK_MARG",
                "WALK_TO_WORK_EST",
                "WALK_TO_WORK_MARG",
                "DRIVE_SOLO_TO_WORK_EST",
                "DRIVE_SOLO_TO_WORK_MARG",
                "CARPOOL_TO_WORK_EST",
                "CARPOOL_TO_WORK_MARG",
                "PUBTRANS_TO_WORK_EST",
                "PUBTRANS_TO_WORK_MARG",
                "state",                 
                "county",                
                "county_subdivision"]] = ACS_MUNI_DF[["POPULATION",
                                            "BIKE_TO_WORK_EST",
                                            "BIKE_TO_WORK_MARG",
                                            "WALK_TO_WORK_EST",
                                            "WALK_TO_WORK_MARG",
                                            "DRIVE_SOLO_TO_WORK_EST",
                                            "DRIVE_SOLO_TO_WORK_MARG",
                                            "CARPOOL_TO_WORK_EST",
                                            "CARPOOL_TO_WORK_MARG",
                                            "PUBTRANS_TO_WORK_EST",
                                            "PUBTRANS_TO_WORK_MARG",
                                            "state",                 
                                            "county",                
                                            "county_subdivision"]].astype(int)

# Show the length of an object referred to the dataframe
print(len(ACS_MUNI_DF))

# Show the dataframe
ACS_MUNI_DF.head()

357


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
0,"County subdivisions not defined, Barnstable Co...",0,0,13,0,13,0,13,0,13,0,13,25,1,0,County subdivisions not defined,Barnstable County
1,"Barnstable Town city, Barnstable County",48556,35,52,843,321,18901,926,2649,482,303,136,25,1,3690,Barnstable Town city,Barnstable County
2,"Bourne town, Barnstable County",20364,0,25,182,64,8471,673,722,288,84,55,25,1,7175,Bourne town,Barnstable County
3,"Brewster town, Barnstable County",10282,66,70,1,3,3733,445,153,88,24,29,25,1,7980,Brewster town,Barnstable County
4,"Chatham town, Barnstable County",6554,0,19,124,74,1666,252,196,96,93,102,25,1,12995,Chatham town,Barnstable County


## Importing Massachusetts Person Level Crash Data from 2017 to 2021

**The following is my code:**

In [6]:
# Import datasets for the years 2017 to 2021
for num in range(2017, 2022):
    if num == 2018 or num == 2019:
        filename = f'./data/{num}_Person_Level_Details.csv'
    else:
        filename = f'./data/{num}_Person_Level_Crash_Details.csv'
    
    # Read the CSV file into a DataFrame and assign it to a dynamically named variable
    var_name = f'crash_{num}'  # Create a variable name
    
    # globals() provides access to variables defined at the top level of your script or module
    globals()[var_name] = pd.read_csv(filename)
    
    # Can also use locals for this cell because locals() provides access to variables defined within the current function or scope
    # locals()[var_name] = pd.read_csv(filename)

/var/folders/vw/dbzntfg96b58pmmcc2cxykdh0000gn/T/ipykernel_7418/1070486101.py:12: DtypeWarning: Columns (25,29,30,32,33,67) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(filename)
/var/folders/vw/dbzntfg96b58pmmcc2cxykdh0000gn/T/ipykernel_7418/1070486101.py:12: DtypeWarning: Columns (25,30,33,67,68,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(filename)
/var/folders/vw/dbzntfg96b58pmmcc2cxykdh0000gn/T/ipykernel_7418/1070486101.py:12: DtypeWarning: Columns (25,33,34,67,68,96,108,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(filename)
/var/folders/vw/dbzntfg96b58pmmcc2cxykdh0000gn/T/ipykernel_7418/1070486101.py:12: DtypeWarning: Columns (24,29,32,33,66) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(filename)
/var/folde

## Data Cleaning

In [7]:
# Show the difference of variables between datasets
# Create a list of dataset names for the years 2018 to 2021
dataset_years = [2018, 2019, 2020, 2021]

# Loop through the list of dataset years
for year in dataset_years:
    # Generate the dataset variable names based on the year
    current_dataset_name = f'crash_{year}'
    
    # Find the columns that are unique to the current year's dataset compared to 2017
    columns_only_in_current = set(locals()[current_dataset_name].columns) - set(crash_2017.columns)
    
    if columns_only_in_current:
        print(f"\nColumns only in {current_dataset_name}:")
        for column in columns_only_in_current:
            print(column)


Columns only in crash_2018:
CRASH_TIME_2
CRASH_DATE
T_EXC_TIME
T_EXC_TYPE

Columns only in crash_2019:
CRASH_TIME_2
CRASH_DATE
T_EXC_TIME
T_EXC_TYPE

Columns only in crash_2020:
CRASH_TIME_2
CRASH_DATE
T_EXC_TIME
T_EXC_TYPE

Columns only in crash_2021:
CRASH_TIME_2
TRVL_DIRC_DESCR
T_EXC_TIME
SHAPE
T_EXC_TYPE


In [8]:
# Rename the columns
# Keep 'CRASH_DATE' and 'CRASH_TIME_2' since they contain the same type of information as 'CRASH_DATETIME' and 'CRASH_TIME' in 'crash_2017'
crash_2018.rename(columns={'CRASH_TIME_2': 'CRASH_TIME', 'CRASH_DATE': 'CRASH_DATETIME'}, inplace=True)
crash_2019.rename(columns={'CRASH_TIME_2': 'CRASH_TIME', 'CRASH_DATE': 'CRASH_DATETIME'}, inplace=True)
crash_2020.rename(columns={'CRASH_TIME_2': 'CRASH_TIME', 'CRASH_DATE': 'CRASH_DATETIME'}, inplace=True)
crash_2021.rename(columns={'CRASH_TIME_2': 'CRASH_TIME'}, inplace=True)

# Drop columns which aren't in crash_2017
crash_2018 = crash_2018.drop(columns = ['T_EXC_TYPE', 'T_EXC_TIME'])
crash_2019 = crash_2019.drop(columns = ['T_EXC_TYPE', 'T_EXC_TIME'])
crash_2020 = crash_2020.drop(columns = ['T_EXC_TYPE', 'T_EXC_TIME'])
crash_2021 = crash_2021.drop(columns = ['SHAPE', 'TRVL_DIRC_DESCR', 'T_EXC_TYPE', 'T_EXC_TIME'])

# List of dataset names for the years 2018 to 2021
dataset_years = [2018, 2019, 2020, 2021]

# Loop through the list of dataset years
for year in dataset_years:
    # Generate the dataset variable name based on the year
    current_dataset_name = f'crash_{year}'
    
    # Reorder the columns of the current dataset to match crash_2017
    locals()[current_dataset_name] = locals()[current_dataset_name][crash_2017.columns]

In [9]:
# Check the shape and columns (variables) of the datasets
# List of dataset names for the years 2017 to 2021
years = [2017, 2018, 2019, 2020, 2021]

# Loop through the list of dataset years
for year in years:
    # Generate the dataset variable name based on the year
    current_dataset_name = f'crash_{year}'
    
    # Check the shape of the current dataset
    dataset_shape = locals()[current_dataset_name].shape # locals() provides access to variables defined within the current function or scope
    
    # Get the columns of the current dataset
    dataset_columns = locals()[current_dataset_name].columns
    
    # Print the results
    print(f"Year {year}:")
    print(f"Shape: {dataset_shape}")
    print(f"Columns: {dataset_columns}")
    print("\n")

Year 2017:
Shape: (337569, 154)
Columns: Index(['OBJECTID', 'CRASH_NUMB', 'CITY_TOWN_NAME', 'CRASH_DATE_TEXT', 'CRASH_TIME', 'CRASH_DATETIME', 'CRASH_HOUR', 'CRASH_STATUS', 'CRASH_SEVERITY_DESCR', 'MAX_INJR_SVRTY_CL',
       ...
       'NON_MTRST_LOC_DESCR', 'NON_MTRST_TYPE_DESCR', 'PERS_ADDR_CITY', 'STATE_PRVN_CODE', 'PERS_TYPE', 'PRTC_SYS_USE_DESCR', 'SFTY_EQUP_DESC_1', 'SFTY_EQUP_DESC_2', 'SEX_DESCR', 'TRNSD_BY_DESCR'], dtype='object', length=154)


Year 2018:
Shape: (330805, 154)
Columns: Index(['OBJECTID', 'CRASH_NUMB', 'CITY_TOWN_NAME', 'CRASH_DATE_TEXT', 'CRASH_TIME', 'CRASH_DATETIME', 'CRASH_HOUR', 'CRASH_STATUS', 'CRASH_SEVERITY_DESCR', 'MAX_INJR_SVRTY_CL',
       ...
       'NON_MTRST_LOC_DESCR', 'NON_MTRST_TYPE_DESCR', 'PERS_ADDR_CITY', 'STATE_PRVN_CODE', 'PERS_TYPE', 'PRTC_SYS_USE_DESCR', 'SFTY_EQUP_DESC_1', 'SFTY_EQUP_DESC_2', 'SEX_DESCR', 'TRNSD_BY_DESCR'], dtype='object', length=154)


Year 2019:
Shape: (325406, 154)
Columns: Index(['OBJECTID', 'CRASH_NUMB', 'CITY_TOWN_N

In [10]:
# Define an empty dataframe that will be populated
mass_crash = pd.DataFrame()

# Create a list of these DataFrames
dataframes = [crash_2017, crash_2018, crash_2019, crash_2020, crash_2021]

# Concatenate them into a single DataFrame
mass_crash = pd.concat(dataframes, ignore_index=True)

# Reset the index
mass_crash = mass_crash.reset_index(drop=True)

# Print the length of mass_crash
print(len(mass_crash))

# Show mass_crash
mass_crash.head()

1498557


,OBJECTID,CRASH_NUMB,CITY_TOWN_NAME,CRASH_DATE_TEXT,CRASH_TIME,CRASH_DATETIME,CRASH_HOUR,CRASH_STATUS,CRASH_SEVERITY_DESCR,MAX_INJR_SVRTY_CL,NUMB_VEHC,NUMB_NONFATAL_INJR,NUMB_FATAL_INJR,POLC_AGNCY_TYPE_DESCR,MANR_COLL_DESCR,VEHC_MNVR_ACTN_CL,VEHC_TRVL_DIRC_CL,VEHC_SEQ_EVENTS_CL,AMBNT_LIGHT_DESCR,WEATH_COND_DESCR,ROAD_SURF_COND_DESCR,FIRST_HRMF_EVENT_DESCR,MOST_HRMFL_EVT_CL,DRVR_CNTRB_CIRC_CL,VEHC_CONFIG_CL,STREET_NUMB,RDWY,DIST_DIRC_FROM_INT,NEAR_INT_RDWY,MM_RTE,DIST_DIRC_MILEMARKER,MILEMARKER,EXIT_RTE,DIST_DIRC_EXIT,EXIT_NUMB,DIST_DIRC_LANDMARK,LANDMARK,RDWY_JNCT_TYPE_DESCR,TRAF_CNTRL_DEVC_TYPE_DESCR,TRAFY_DESCR_DESCR,JURISDICTN,FIRST_HRMF_EVENT_LOC_DESCR,NON_MTRST_TYPE_CL,NON_MTRST_ACTN_CL,NON_MTRST_LOC_CL,IS_GEOCODED,GEOCODING_METHOD_NAME,X,Y,LAT,LON,RMV_DOC_IDS,CRASH_RPT_IDS,YEAR,AGE_DRVR_YNGST,AGE_DRVR_OLDEST,AGE_NONMTRST_YNGST,AGE_NONMTRST_OLDEST,DRVR_DISTRACTED_CL,DISTRICT_NUM,RPA_ABBR,VEHC_EMER_USE_CL,VEHC_TOWED_FROM_SCENE_CL,CNTY_NAME,FMCSA_RPTBL_CL,FMCSA_RPTBL,HIT_RUN_DESCR,LCLTY_NAME,ROAD_CNTRB_DESCR,SCHL_BUS_RELD_DESCR,SPEED_LIMIT,TRAF_CNTRL_DEVC_FUNC_DESCR,WORK_ZONE_RELD_DESCR,AADT,AADT_YEAR,PK_PCT_SUT,AV_PCT_SUT,PK_PCT_CT,AV_PCT_CT,CURB,TRUCK_RTE,LT_SIDEWLK,RT_SIDEWLK,SHLDR_LT_W,SHLDR_LT_T,SURFACE_WD,SURFACE_TP,SHLDR_RT_W,SHLDR_RT_T,NUM_LANES,OPP_LANES,MED_WIDTH,MED_TYPE,URBAN_TYPE,F_CLASS,URBAN_AREA,FD_AID_RTE,FACILITY,OPERATION,CONTROL,PEAK_LANE,SPEED_LIM,STREETNAME,FROMSTREETNAME,TOSTREETNAME,CITY,STRUCT_CND,TERRAIN,URBAN_LOC_TYPE,AADT_DERIV,STATN_NUM,OP_DIR_SL,SHLDR_UL_T,SHLDR_UL_W,F_F_CLASS,VEHC_UNIT_NUMB,ALC_SUSPD_TYPE_DESCR,DRIVER_AGE,DRVR_CNTRB_CIRC_DESCR,DRIVER_DISTRACTED_TYPE_DESCR,DRVR_LCN_STATE,DRUG_SUSPD_TYPE_DESCR,EMERGENCY_USE_DESC,FMCSA_RPTBL_VL,HAZ_MAT_PLACARD_DESCR,MAX_INJR_SVRTY_VL,MOST_HRMF_EVENT,TOTAL_OCCPT_IN_VEHC,VEHC_MANR_ACT_DESCR,VEHC_CONFG_DESCR,VEHC_MOST_DMGD_AREA,OWNER_ADDR_CITY_TOWN,OWNER_ADDR_STATE,VEHC_REG_STATE,VEHC_REG_TYPE_CODE,VEHC_SEQ_EVENTS,VEHC_TOWED_FROM_SCENE,PERS_NUMB,AGE,EJCTN_DESCR,INJY_STAT_DESCR,MED_FACLY,NON_MTRST_ACT_DESCR,NON_MTRST_COND_DESCR,NON_MTRST_LOC_DESCR,NON_MTRST_TYPE_DESCR,PERS_ADDR_CITY,STATE_PRVN_CODE,PERS_TYPE,PRTC_SYS_USE_DESCR,SFTY_EQUP_DESC_1,SFTY_EQUP_DESC_2,SEX_DESCR,TRNSD_BY_DESCR
0,1,4309633,WILBRAHAM,01/02/2017,12:50 PM,2017/01/02 12:49:59+00,12:00PM to 12:59PM,Closed,Property damage only (none injured),No injury,2,0,0,Local police,Angle,V1: Entering traffic lane / V2: Travelling str...,V1: N / V2: W,V1:(Collision with motor vehicle in traffic) ...,Daylight,Cloudy,Dry,Collision with motor vehicle in traffic,V1:(Collision with motor vehicle in traffic) /...,D1: (Unknown) / D2: (Unknown),V1:(Passenger car) / V2:(Passenger car),2001,BOSTON RD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME DEPOT EXIT/ENTRY,Four-way intersection,Traffic control signal,"Two-way, not divided",Massachusetts Department of Transportation,Roadway,NaN,NaN,NaN,Yes,At Address,"119,450.357177","878,131.459214",42.149468,-72.474569,PW201701201142,17-3-AC,2017,65-74,>84,NaN,NaN,NaN,2,PVPC,V1:(No) / V2:(No),V1:(No) / V2:(No),HAMPDEN,NaN,NaN,No hit and run,NaN,None,"No, school bus not involved",NaN,"Yes, device functioning",No,"17,338.000000","2,013.000000",0.613000,"1,016.000000",0.121000,237.000000,Both sides,Designated truck route ONLY under State Author...,4.000000,4.000000,0.000000,No Shoulder,36.000000,Bituminous concrete road,0.000000,No Shoulder,3.000000,0.000000,0.000000,None,Large Urbanized Area,Rural minor arterial or urban principal arterial,Springfield (MA-CT),NaN,Mainline roadway,Two-way traffic,No control,NaN,35.000000,BOSTON ROAD,SPRINGFIELD CITY LINE,PALMER TOWN LINE,Wilbraham,Good,Level,Low density commercial,NaN,NaN,NaN,NaN,NaN,Principal Arterial - Other,2.000000,NaN,86.000000,"DCC1:Unknown, DCC2:Not reported",Unknown,MA,NaN,No,NaN,Not reported,No injury,Collision with motor vehicle in traffic,1.000000,Travelling straight ahead,Passenger car,MDA1:Left side,SPRINGFIELD,MA,MA,PAN,SEQ1:Collision with motor vehicle in traffic,No,2,86.000000,Not ejected,No injury,NaN,NaN,

## Exploratory Data Analysis (EDA)

In [11]:
# Exclude the NaN from 'NON_MTRST_TYPE_CL' (Non-Motorist Type) and 'INJY_STAT_DESCR' (Injury Status) columns
df_analysis = mass_crash[(mass_crash['NON_MTRST_TYPE_CL'].notna()) & (mass_crash['INJY_STAT_DESCR'].notna())]

# Reset the index of df_analysis
df_analysis.reset_index(drop=True, inplace=True)

# Make a copy of it to fix the error
df_analysis = df_analysis.copy()

# Clean data
df_analysis['NON_MTRST_TYPE_CL'] = df_analysis['NON_MTRST_TYPE_CL'].str.split(':').str[1] # Non-Motorist Type
df_analysis['NON_MTRST_TYPE_CL'] = df_analysis['NON_MTRST_TYPE_CL'].str.strip()

# Show dataframe
df_analysis.head(3)

,OBJECTID,CRASH_NUMB,CITY_TOWN_NAME,CRASH_DATE_TEXT,CRASH_TIME,CRASH_DATETIME,CRASH_HOUR,CRASH_STATUS,CRASH_SEVERITY_DESCR,MAX_INJR_SVRTY_CL,NUMB_VEHC,NUMB_NONFATAL_INJR,NUMB_FATAL_INJR,POLC_AGNCY_TYPE_DESCR,MANR_COLL_DESCR,VEHC_MNVR_ACTN_CL,VEHC_TRVL_DIRC_CL,VEHC_SEQ_EVENTS_CL,AMBNT_LIGHT_DESCR,WEATH_COND_DESCR,ROAD_SURF_COND_DESCR,FIRST_HRMF_EVENT_DESCR,MOST_HRMFL_EVT_CL,DRVR_CNTRB_CIRC_CL,VEHC_CONFIG_CL,STREET_NUMB,RDWY,DIST_DIRC_FROM_INT,NEAR_INT_RDWY,MM_RTE,DIST_DIRC_MILEMARKER,MILEMARKER,EXIT_RTE,DIST_DIRC_EXIT,EXIT_NUMB,DIST_DIRC_LANDMARK,LANDMARK,RDWY_JNCT_TYPE_DESCR,TRAF_CNTRL_DEVC_TYPE_DESCR,TRAFY_DESCR_DESCR,JURISDICTN,FIRST_HRMF_EVENT_LOC_DESCR,NON_MTRST_TYPE_CL,NON_MTRST_ACTN_CL,NON_MTRST_LOC_CL,IS_GEOCODED,GEOCODING_METHOD_NAME,X,Y,LAT,LON,RMV_DOC_IDS,CRASH_RPT_IDS,YEAR,AGE_DRVR_YNGST,AGE_DRVR_OLDEST,AGE_NONMTRST_YNGST,AGE_NONMTRST_OLDEST,DRVR_DISTRACTED_CL,DISTRICT_NUM,RPA_ABBR,VEHC_EMER_USE_CL,VEHC_TOWED_FROM_SCENE_CL,CNTY_NAME,FMCSA_RPTBL_CL,FMCSA_RPTBL,HIT_RUN_DESCR,LCLTY_NAME,ROAD_CNTRB_DESCR,SCHL_BUS_RELD_DESCR,SPEED_LIMIT,TRAF_CNTRL_DEVC_FUNC_DESCR,WORK_ZONE_RELD_DESCR,AADT,AADT_YEAR,PK_PCT_SUT,AV_PCT_SUT,PK_PCT_CT,AV_PCT_CT,CURB,TRUCK_RTE,LT_SIDEWLK,RT_SIDEWLK,SHLDR_LT_W,SHLDR_LT_T,SURFACE_WD,SURFACE_TP,SHLDR_RT_W,SHLDR_RT_T,NUM_LANES,OPP_LANES,MED_WIDTH,MED_TYPE,URBAN_TYPE,F_CLASS,URBAN_AREA,FD_AID_RTE,FACILITY,OPERATION,CONTROL,PEAK_LANE,SPEED_LIM,STREETNAME,FROMSTREETNAME,TOSTREETNAME,CITY,STRUCT_CND,TERRAIN,URBAN_LOC_TYPE,AADT_DERIV,STATN_NUM,OP_DIR_SL,SHLDR_UL_T,SHLDR_UL_W,F_F_CLASS,VEHC_UNIT_NUMB,ALC_SUSPD_TYPE_DESCR,DRIVER_AGE,DRVR_CNTRB_CIRC_DESCR,DRIVER_DISTRACTED_TYPE_DESCR,DRVR_LCN_STATE,DRUG_SUSPD_TYPE_DESCR,EMERGENCY_USE_DESC,FMCSA_RPTBL_VL,HAZ_MAT_PLACARD_DESCR,MAX_INJR_SVRTY_VL,MOST_HRMF_EVENT,TOTAL_OCCPT_IN_VEHC,VEHC_MANR_ACT_DESCR,VEHC_CONFG_DESCR,VEHC_MOST_DMGD_AREA,OWNER_ADDR_CITY_TOWN,OWNER_ADDR_STATE,VEHC_REG_STATE,VEHC_REG_TYPE_CODE,VEHC_SEQ_EVENTS,VEHC_TOWED_FROM_SCENE,PERS_NUMB,AGE,EJCTN_DESCR,INJY_STAT_DESCR,MED_FACLY,NON_MTRST_ACT_DESCR,NON_MTRST_COND_DESCR,NON_MTRST_LOC_DESCR,NON_MTRST_TYPE_DESCR,PERS_ADDR_CITY,STATE_PRVN_CODE,PERS_TYPE,PRTC_SYS_USE_DESCR,SFTY_EQUP_DESC_1,SFTY_EQUP_DESC_2,SEX_DESCR,TRNSD_BY_DESCR
0,118,4323882,BOSTON,01/18/2017,6:38 PM,2017/01/18 18:37:59+00,06:00PM to 06:59PM,Closed,Property damage only (none injured),No injury,1,0,0,State police,Single vehicle crash,V1: Turning left,V1: W,V1:(Collision with pedestrian),Dark - lighted roadway,Cloudy,Dry,Collision with motor vehicle in traffic,V1:(Collision with pedestrian),D1: (Failed to yield right of way),V1:(Passenger car),NaN,NEPONSET VALLEY PARKWAY Rte UNKNOW,NaN,MILTON STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T-intersection,Stop signs,"Two-way, not divided",Department of Conservation and Recreation,Roadway,Pedestrian,P2: Entering or crossing specified location,P2: Marked crosswalk at intersection,Yes,Off Intersection,"230,226.968526","887,751.687407",42.239645,-71.133772,PW201704401255,2017-0H7-000369,2017,35-44,35-44,35-44,35-44,NaN,6,MAPC,V1:(No),NaN,SUFFOLK,NaN,NaN,No hit and run,NaN,None,"No, school bus not involved",25.000000,"Yes, device functioning",No,"9,342.000000","2,013.000000",0.304000,261.000000,0.100000,98.000000,Both sides,Not a parkway - not on a designated truck route,NaN,6.000000,NaN,NaN,30.000000,Bituminous concrete road,0.000000,No Shoulder,2.000000,0.000000,NaN,None,Large Urbanized Area,Urban minor arterial or rural major collector,Boston (MA-NH-RI),NaN,Mainline roadway,Two-way traffic,No control,NaN,NaN,NEPONSET VALLEY PARKWAY,RIVER STREET,MILTON STREET,Boston,Fair,Rolling,Low density commercial,NaN,NaN,NaN,NaN,NaN,Minor Arterial,1.000000,NaN,43.000000,"DCC1:Failed to yield right of way, DCC2:Not re...",Not Distracted,MA,NaN,No,NaN,Not reported,No injury,Collision with pedestrian,1.000000,Turning left,Passenger car,MDA1:None,BROCKTON,MA,MA,PAN,SEQ1:Collision with pedestrian,Not reported,1,43.000000,Not ejected,No injury,NaN,NaN,NaN,NaN,NaN,BROCKTON,MA,Driver,Shoulder and lap belt used,

In [12]:
# Fatal - injuries that resulted in death 
# Incapacitating - serious injuries require immediate medical attention

# The number of fatal injuries to cyclists
num_fatal_cyclist = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Cyclist')].shape[0]
print('The number of fatal injuries to cyclists: ' + str(num_fatal_cyclist))

# Include pedestrians and skaters together since PennDOT does not discriminate between the two

# The number of fatal injuries to pedestrians and skaters
num_fatal_pedestrian = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Pedestrain')].shape[0]
num_fatal_skater = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Skater')].shape[0]
fatal_pedestrian_skater = num_fatal_pedestrian + num_fatal_skater
print('The number of fatal injuries to pedestrians and skaters: ' + str(fatal_pedestrian_skater))

# The number of incapacitating injuries to cyclists
num_incapacitating_cyclist = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Cyclist')].shape[0]
print('The number of incapacitating injuries to cyclists: ' + str(num_incapacitating_cyclist))

# The number of incapacitating injuries to pedestrians and skaters
num_incapacitating_pedestrian = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Pedestrain')].shape[0]
num_incapacitating_skater = df_analysis[(df_analysis['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_analysis['NON_MTRST_TYPE_CL'] == 'Skater')].shape[0]
incapacitating_pedestrian_skater = num_incapacitating_pedestrian + num_incapacitating_skater
print('The number of incapacitating injuries to pedestrians and skaters: ' + str(fatal_pedestrian_skater))

The number of fatal injuries to cyclists: 33
The number of fatal injuries to pedestrians and skaters: 0
The number of incapacitating injuries to cyclists: 217
The number of incapacitating injuries to pedestrians and skaters: 0


## Merge ACS Data and mass_crash Data (Massachusetts Personal Level Crash Data from 2017 to 2021)¶

In [13]:
# Clean ACS data
# Define an empty dataframe that will be populated
df_acs_muni = pd.DataFrame()

# Create a copy of ACS_MUNI_DF as df_acs_muni
df_acs_muni = ACS_MUNI_DF.copy()

# Create a boolean mask to identify rows where 'MUNI_NAME' is 'County subdivisions not defined'
mask = df_acs_muni['MUNI_NAME'] == 'County subdivisions not defined'

# Use the mask to drop rows from the DataFrame
df_acs_muni = df_acs_muni[~mask]

# Reset the index and use the drop parameter to remove the old index column
df_acs_muni.reset_index(drop=True, inplace=True)

# Print the shape of dataframe 
print(df_acs_muni.shape)

# Show dataframe
df_acs_muni.head()

(351, 17)


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
0,"Barnstable Town city, Barnstable County",48556,35,52,843,321,18901,926,2649,482,303,136,25,1,3690,Barnstable Town city,Barnstable County
1,"Bourne town, Barnstable County",20364,0,25,182,64,8471,673,722,288,84,55,25,1,7175,Bourne town,Barnstable County
2,"Brewster town, Barnstable County",10282,66,70,1,3,3733,445,153,88,24,29,25,1,7980,Brewster town,Barnstable County
3,"Chatham town, Barnstable County",6554,0,19,124,74,1666,252,196,96,93,102,25,1,12995,Chatham town,Barnstable County
4,"Dennis town, Barnstable County",14664,8,16,147,108,4930,515,658,173,37,47,25,1,16775,Dennis town,Barnstable County


In [14]:
# Lower the muni name from acs dataframe
df_acs_muni["muni"] = df_acs_muni["MUNI_NAME"].str.lower()

# Split the muni_name from the acs dataframe for merging
df_acs_muni["municipality_name"] = df_acs_muni["muni"].str.split(' ').str[0]
df_acs_muni["municipality_name_second"] = df_acs_muni["muni"].str.split(' ').str[1]

# Define a function to conditionally concatenate columns
def concatenate_municipalities(row):
    if "town" not in row["municipality_name_second"] and "city" not in row["municipality_name_second"]:
        return row["municipality_name"] + " " + row["municipality_name_second"]
    else:
        return row["municipality_name"]

# Apply the function to create a new column
df_acs_muni["muni_name"] = df_acs_muni.apply(concatenate_municipalities, axis=1)

# Drop unnecessary columns
df_acs_muni = df_acs_muni.drop(columns=["muni","municipality_name", "municipality_name_second"])

In [15]:
# Show dataframe
df_acs_muni.head(3)

,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME,muni_name
0,"Barnstable Town city, Barnstable County",48556,35,52,843,321,18901,926,2649,482,303,136,25,1,3690,Barnstable Town city,Barnstable County,barnstable
1,"Bourne town, Barnstable County",20364,0,25,182,64,8471,673,722,288,84,55,25,1,7175,Bourne town,Barnstable County,bourne
2,"Brewster town, Barnstable County",10282,66,70,1,3,3733,445,153,88,24,29,25,1,7980,Brewster town,Barnstable County,brewster


In [16]:
# Make a copy of df_analysis which excludes NaN values of Non-Motorist Types and clean the values of that column
mass_crash_df = df_analysis.copy()

# Keep the rows where 'NON_MTRST_TYPE_CL' is in the list ['Cyclist', 'Pedestrian', 'Skater']
mass_crash_df = mass_crash_df[mass_crash_df['NON_MTRST_TYPE_CL'].isin(['Cyclist', 'Pedestrian', 'Skater'])]

# Reset index
mass_crash_df.reset_index(drop=True, inplace=True)

# Show mass_crash_df
mass_crash_df.head(3)

,OBJECTID,CRASH_NUMB,CITY_TOWN_NAME,CRASH_DATE_TEXT,CRASH_TIME,CRASH_DATETIME,CRASH_HOUR,CRASH_STATUS,CRASH_SEVERITY_DESCR,MAX_INJR_SVRTY_CL,NUMB_VEHC,NUMB_NONFATAL_INJR,NUMB_FATAL_INJR,POLC_AGNCY_TYPE_DESCR,MANR_COLL_DESCR,VEHC_MNVR_ACTN_CL,VEHC_TRVL_DIRC_CL,VEHC_SEQ_EVENTS_CL,AMBNT_LIGHT_DESCR,WEATH_COND_DESCR,ROAD_SURF_COND_DESCR,FIRST_HRMF_EVENT_DESCR,MOST_HRMFL_EVT_CL,DRVR_CNTRB_CIRC_CL,VEHC_CONFIG_CL,STREET_NUMB,RDWY,DIST_DIRC_FROM_INT,NEAR_INT_RDWY,MM_RTE,DIST_DIRC_MILEMARKER,MILEMARKER,EXIT_RTE,DIST_DIRC_EXIT,EXIT_NUMB,DIST_DIRC_LANDMARK,LANDMARK,RDWY_JNCT_TYPE_DESCR,TRAF_CNTRL_DEVC_TYPE_DESCR,TRAFY_DESCR_DESCR,JURISDICTN,FIRST_HRMF_EVENT_LOC_DESCR,NON_MTRST_TYPE_CL,NON_MTRST_ACTN_CL,NON_MTRST_LOC_CL,IS_GEOCODED,GEOCODING_METHOD_NAME,X,Y,LAT,LON,RMV_DOC_IDS,CRASH_RPT_IDS,YEAR,AGE_DRVR_YNGST,AGE_DRVR_OLDEST,AGE_NONMTRST_YNGST,AGE_NONMTRST_OLDEST,DRVR_DISTRACTED_CL,DISTRICT_NUM,RPA_ABBR,VEHC_EMER_USE_CL,VEHC_TOWED_FROM_SCENE_CL,CNTY_NAME,FMCSA_RPTBL_CL,FMCSA_RPTBL,HIT_RUN_DESCR,LCLTY_NAME,ROAD_CNTRB_DESCR,SCHL_BUS_RELD_DESCR,SPEED_LIMIT,TRAF_CNTRL_DEVC_FUNC_DESCR,WORK_ZONE_RELD_DESCR,AADT,AADT_YEAR,PK_PCT_SUT,AV_PCT_SUT,PK_PCT_CT,AV_PCT_CT,CURB,TRUCK_RTE,LT_SIDEWLK,RT_SIDEWLK,SHLDR_LT_W,SHLDR_LT_T,SURFACE_WD,SURFACE_TP,SHLDR_RT_W,SHLDR_RT_T,NUM_LANES,OPP_LANES,MED_WIDTH,MED_TYPE,URBAN_TYPE,F_CLASS,URBAN_AREA,FD_AID_RTE,FACILITY,OPERATION,CONTROL,PEAK_LANE,SPEED_LIM,STREETNAME,FROMSTREETNAME,TOSTREETNAME,CITY,STRUCT_CND,TERRAIN,URBAN_LOC_TYPE,AADT_DERIV,STATN_NUM,OP_DIR_SL,SHLDR_UL_T,SHLDR_UL_W,F_F_CLASS,VEHC_UNIT_NUMB,ALC_SUSPD_TYPE_DESCR,DRIVER_AGE,DRVR_CNTRB_CIRC_DESCR,DRIVER_DISTRACTED_TYPE_DESCR,DRVR_LCN_STATE,DRUG_SUSPD_TYPE_DESCR,EMERGENCY_USE_DESC,FMCSA_RPTBL_VL,HAZ_MAT_PLACARD_DESCR,MAX_INJR_SVRTY_VL,MOST_HRMF_EVENT,TOTAL_OCCPT_IN_VEHC,VEHC_MANR_ACT_DESCR,VEHC_CONFG_DESCR,VEHC_MOST_DMGD_AREA,OWNER_ADDR_CITY_TOWN,OWNER_ADDR_STATE,VEHC_REG_STATE,VEHC_REG_TYPE_CODE,VEHC_SEQ_EVENTS,VEHC_TOWED_FROM_SCENE,PERS_NUMB,AGE,EJCTN_DESCR,INJY_STAT_DESCR,MED_FACLY,NON_MTRST_ACT_DESCR,NON_MTRST_COND_DESCR,NON_MTRST_LOC_DESCR,NON_MTRST_TYPE_DESCR,PERS_ADDR_CITY,STATE_PRVN_CODE,PERS_TYPE,PRTC_SYS_USE_DESCR,SFTY_EQUP_DESC_1,SFTY_EQUP_DESC_2,SEX_DESCR,TRNSD_BY_DESCR
0,118,4323882,BOSTON,01/18/2017,6:38 PM,2017/01/18 18:37:59+00,06:00PM to 06:59PM,Closed,Property damage only (none injured),No injury,1,0,0,State police,Single vehicle crash,V1: Turning left,V1: W,V1:(Collision with pedestrian),Dark - lighted roadway,Cloudy,Dry,Collision with motor vehicle in traffic,V1:(Collision with pedestrian),D1: (Failed to yield right of way),V1:(Passenger car),NaN,NEPONSET VALLEY PARKWAY Rte UNKNOW,NaN,MILTON STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T-intersection,Stop signs,"Two-way, not divided",Department of Conservation and Recreation,Roadway,Pedestrian,P2: Entering or crossing specified location,P2: Marked crosswalk at intersection,Yes,Off Intersection,"230,226.968526","887,751.687407",42.239645,-71.133772,PW201704401255,2017-0H7-000369,2017,35-44,35-44,35-44,35-44,NaN,6,MAPC,V1:(No),NaN,SUFFOLK,NaN,NaN,No hit and run,NaN,None,"No, school bus not involved",25.000000,"Yes, device functioning",No,"9,342.000000","2,013.000000",0.304000,261.000000,0.100000,98.000000,Both sides,Not a parkway - not on a designated truck route,NaN,6.000000,NaN,NaN,30.000000,Bituminous concrete road,0.000000,No Shoulder,2.000000,0.000000,NaN,None,Large Urbanized Area,Urban minor arterial or rural major collector,Boston (MA-NH-RI),NaN,Mainline roadway,Two-way traffic,No control,NaN,NaN,NEPONSET VALLEY PARKWAY,RIVER STREET,MILTON STREET,Boston,Fair,Rolling,Low density commercial,NaN,NaN,NaN,NaN,NaN,Minor Arterial,1.000000,NaN,43.000000,"DCC1:Failed to yield right of way, DCC2:Not re...",Not Distracted,MA,NaN,No,NaN,Not reported,No injury,Collision with pedestrian,1.000000,Turning left,Passenger car,MDA1:None,BROCKTON,MA,MA,PAN,SEQ1:Collision with pedestrian,Not reported,1,43.000000,Not ejected,No injury,NaN,NaN,NaN,NaN,NaN,BROCKTON,MA,Driver,Shoulder and lap belt used,

In [17]:
# Lower the CITY_TOWN_NAME name from mass_crash dataframe
mass_crash_df["muni_name"] = mass_crash_df["CITY_TOWN_NAME"].str.lower()

# Define an empty dataframe that will be populated
df_mass_acs = pd.DataFrame()

# Merge dataframes
df_mass_acs = mass_crash_df.merge(df_acs_muni, how="inner", on="muni_name")

# Drop the muni_name column
df_mass_acs = df_mass_acs.drop(columns=["muni_name"])

# Reset the index
df_mass_acs = df_mass_acs.reset_index(drop=True)

# Show dataframe
df_mass_acs.head()

,OBJECTID,CRASH_NUMB,CITY_TOWN_NAME,CRASH_DATE_TEXT,CRASH_TIME,CRASH_DATETIME,CRASH_HOUR,CRASH_STATUS,CRASH_SEVERITY_DESCR,MAX_INJR_SVRTY_CL,NUMB_VEHC,NUMB_NONFATAL_INJR,NUMB_FATAL_INJR,POLC_AGNCY_TYPE_DESCR,MANR_COLL_DESCR,VEHC_MNVR_ACTN_CL,VEHC_TRVL_DIRC_CL,VEHC_SEQ_EVENTS_CL,AMBNT_LIGHT_DESCR,WEATH_COND_DESCR,ROAD_SURF_COND_DESCR,FIRST_HRMF_EVENT_DESCR,MOST_HRMFL_EVT_CL,DRVR_CNTRB_CIRC_CL,VEHC_CONFIG_CL,STREET_NUMB,RDWY,DIST_DIRC_FROM_INT,NEAR_INT_RDWY,MM_RTE,DIST_DIRC_MILEMARKER,MILEMARKER,EXIT_RTE,DIST_DIRC_EXIT,EXIT_NUMB,DIST_DIRC_LANDMARK,LANDMARK,RDWY_JNCT_TYPE_DESCR,TRAF_CNTRL_DEVC_TYPE_DESCR,TRAFY_DESCR_DESCR,JURISDICTN,FIRST_HRMF_EVENT_LOC_DESCR,NON_MTRST_TYPE_CL,NON_MTRST_ACTN_CL,NON_MTRST_LOC_CL,IS_GEOCODED,GEOCODING_METHOD_NAME,X,Y,LAT,LON,RMV_DOC_IDS,CRASH_RPT_IDS,YEAR,AGE_DRVR_YNGST,AGE_DRVR_OLDEST,AGE_NONMTRST_YNGST,AGE_NONMTRST_OLDEST,DRVR_DISTRACTED_CL,DISTRICT_NUM,RPA_ABBR,VEHC_EMER_USE_CL,VEHC_TOWED_FROM_SCENE_CL,CNTY_NAME,FMCSA_RPTBL_CL,FMCSA_RPTBL,HIT_RUN_DESCR,LCLTY_NAME,ROAD_CNTRB_DESCR,SCHL_BUS_RELD_DESCR,SPEED_LIMIT,TRAF_CNTRL_DEVC_FUNC_DESCR,WORK_ZONE_RELD_DESCR,AADT,AADT_YEAR,PK_PCT_SUT,AV_PCT_SUT,PK_PCT_CT,AV_PCT_CT,CURB,TRUCK_RTE,LT_SIDEWLK,RT_SIDEWLK,SHLDR_LT_W,SHLDR_LT_T,SURFACE_WD,SURFACE_TP,SHLDR_RT_W,SHLDR_RT_T,NUM_LANES,OPP_LANES,MED_WIDTH,MED_TYPE,URBAN_TYPE,F_CLASS,URBAN_AREA,FD_AID_RTE,FACILITY,OPERATION,CONTROL,PEAK_LANE,SPEED_LIM,STREETNAME,FROMSTREETNAME,TOSTREETNAME,CITY,STRUCT_CND,TERRAIN,URBAN_LOC_TYPE,AADT_DERIV,STATN_NUM,OP_DIR_SL,SHLDR_UL_T,SHLDR_UL_W,F_F_CLASS,VEHC_UNIT_NUMB,ALC_SUSPD_TYPE_DESCR,DRIVER_AGE,DRVR_CNTRB_CIRC_DESCR,DRIVER_DISTRACTED_TYPE_DESCR,DRVR_LCN_STATE,DRUG_SUSPD_TYPE_DESCR,EMERGENCY_USE_DESC,FMCSA_RPTBL_VL,HAZ_MAT_PLACARD_DESCR,MAX_INJR_SVRTY_VL,MOST_HRMF_EVENT,TOTAL_OCCPT_IN_VEHC,VEHC_MANR_ACT_DESCR,VEHC_CONFG_DESCR,VEHC_MOST_DMGD_AREA,OWNER_ADDR_CITY_TOWN,OWNER_ADDR_STATE,VEHC_REG_STATE,VEHC_REG_TYPE_CODE,VEHC_SEQ_EVENTS,VEHC_TOWED_FROM_SCENE,PERS_NUMB,AGE,EJCTN_DESCR,INJY_STAT_DESCR,MED_FACLY,NON_MTRST_ACT_DESCR,NON_MTRST_COND_DESCR,NON_MTRST_LOC_DESCR,NON_MTRST_TYPE_DESCR,PERS_ADDR_CITY,STATE_PRVN_CODE,PERS_TYPE,PRTC_SYS_USE_DESCR,SFTY_EQUP_DESC_1,SFTY_EQUP_DESC_2,SEX_DESCR,TRNSD_BY_DESCR,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
0,118,4323882,BOSTON,01/18/2017,6:38 PM,2017/01/18 18:37:59+00,06:00PM to 06:59PM,Closed,Property damage only (none injured),No injury,1,0,0,State police,Single vehicle crash,V1: Turning left,V1: W,V1:(Collision with pedestrian),Dark - lighted roadway,Cloudy,Dry,Collision with motor vehicle in traffic,V1:(Collision with pedestrian),D1: (Failed to yield right of way),V1:(Passenger car),NaN,NEPONSET VALLEY PARKWAY Rte UNKNOW,NaN,MILTON STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T-intersection,Stop signs,"Two-way, not divided",Department of Conservation and Recreation,Roadway,Pedestrian,P2: Entering or crossing specified location,P2: Marked crosswalk at intersection,Yes,Off Intersection,"230,226.968526","887,751.687407",42.239645,-71.133772,PW201704401255,2017-0H7-000369,2017,35-44,35-44,35-44,35-44,NaN,6,MAPC,V1:(No),NaN,SUFFOLK,NaN,NaN,No hit and run,NaN,None,"No, school bus not involved",25.000000,"Yes, device functioning",No,"9,342.000000","2,013.000000",0.304000,261.000000,0.100000,98.000000,Both sides,Not a parkway - not on a designated truck route,NaN,6.000000,NaN,NaN,30.000000,Bituminous concrete road,0.000000,No Shoulder,2.000000,0.000000,NaN,None,Large Urbanized Area,Urban minor arterial or rural major collector,Boston (MA-NH-RI),NaN,Mainline roadway,Two-way traffic,No control,NaN,NaN,NEPONSET VALLEY PARKWAY,RIVER STREET,MILTON STREET,Boston,Fair,Rolling,Low density commercial,NaN,NaN,NaN,NaN,NaN,Minor Arterial,1.000000,NaN,43.000000,"DCC1:Failed to yield right of way, DCC2:Not re...",Not Distracted,M

In [22]:
# Print the amount of municipalities in df_mass_acs
unique_names = df_mass_acs['CITY_TOWN_NAME'].unique()
print(len(unique_names))

294


In [19]:
# Assuming 'data' is a subdirectory in your current working directory
folder_path = 'data/'
file_name = 'df_mass_acs.csv'

# Combine the folder path and file name to create the full file path
full_file_path = folder_path + file_name

# Export dataframe to csv file
df_mass_acs.to_csv(full_file_path, index=True)

## Compress the CSV file before uploading it to GitHub

In [20]:
import gzip
import shutil

# Path to the CSV file you want to compress
csv_file_path = 'data/df_mass_acs.csv'

# Path for the compressed file
compressed_file_path = 'data/df_mass_acs.csv.gz'

# Open the CSV file for reading
with open(csv_file_path, 'rb') as f_in:
    # Open the compressed file for writing
    with gzip.open(compressed_file_path, 'wb') as f_out:
        # Copy the contents of the CSV file to the compressed file
        shutil.copyfileobj(f_in, f_out)

print(f'File compressed to: {compressed_file_path}')

File compressed to: data/df_mass_acs.csv.gz
